# Mustafas Privet model Zoo LoL


In [ ]:
# %%writefile WillyNet.py

from torch import nn
from torch.nn import functional as F
import torch
# device = torch.device('cuda:0')


def conv(inChannels, outChannels, dilation, groups, dropProbability=0, kernalSize=3):
    """
        Creates the convolution layer that consist of:
        conv2d, batchNorm2d and Mish activation function lastly also a dropout
    """
    return (
        nn.Conv2d(in_channels=inChannels, out_channels=outChannels, kernel_size=kernalSize, stride=1, dilation=dilation, groups=groups
                  ), nn.BatchNorm2d(outChannels), nn.Mish(inplace=True), nn.Dropout2d(dropProbability, inplace=True)
    )


def willy_block(inChannels, outChannels, dilation, groups, dropProbability=0, kernalSize=3):
    #return conv(inChannels,outChannels,dilation,groups,dropProbability=dropProbability,kernalSize=kernalSize)
    return nn.Sequential(*conv(inChannels, outChannels, dilation, groups, dropProbability=dropProbability, kernalSize=kernalSize))


def fullyConnected(inFeatures, outFeatures):
    """
        Creates the standard fully connected layer consist of:
        Linear, barchNorm1d and Mish activation functino
    """
    return (

        # nn.BatchNorm1d(num_features=inFeatures),
        nn.Linear(in_features=inFeatures, out_features=outFeatures),
        nn.BatchNorm1d(outFeatures),
        nn.Mish(inplace=True)
    )


def willy_fullyConnected_block(inFeatures, outFeatures):
    return nn.Sequential(*fullyConnected(inFeatures, outFeatures))


def willy_prediction_block(inFeatures, outFeatures):
    return nn.Sequential(
        nn.Linear(in_features=inFeatures, out_features=outFeatures),
        nn.BatchNorm1d(outFeatures),
        # nn.Mish(inplace=True)
        # F.log_softmax(outFeatures,dim=1)
    )


class WillyNet(nn.Module):
    """
    A CNN designed by Mustafa A-Hussein to predict what category the product is in an image.
    The goal of the design was to satisfy my curiosity and to gain experince in designing Neural Network Architectures
    I also wanted to explore how Mish activation function compares with others such like ReLU.

    Args:
        s (int): how many channel should the first conv2d layer have.
        ToDo add output (int): to decide how many classes you want.
        ToDo add desired shape i.e., how many convolution layer you want with their shape
        ToDo cont: decide how many fully connected layer you want.
    
    """

    def __init__(self, s=20) -> None:  # s = nr_channels_out
        super(WillyNet, self).__init__()

        # the first layer, takes in 3 channel input (RGB) image
        self.conv1 = willy_block(inChannels=3, outChannels=s*9,
                                 dilation=4, groups=3, dropProbability=0.5,
                                 kernalSize=22)

        self.conv2 = willy_block(inChannels=s*9, outChannels=s*5,
                                 dilation=3, groups=4, dropProbability=0.25,
                                 kernalSize=18)

        self.conv3 = willy_block(inChannels=s*5, outChannels=s*5,
                                 dilation=2, groups=2, dropProbability=0.25,
                                 kernalSize=16)

        self.conv4 = willy_block(inChannels=s*5, outChannels=s*3,
                                 dilation=1, groups=1, dropProbability=0.25,
                                 kernalSize=12)

        self.conv5 = willy_block(inChannels=s*3, outChannels=s,
                                 dilation=1, groups=1, dropProbability=0,
                                 kernalSize=8)

        # This part I did do manually until I was satisfied with the archetecture of the model
        self.flatten = nn.Flatten()
        self.fullyConnectedLayer = willy_fullyConnected_block(
            inFeatures=4500, outFeatures=100)
        self.predictionLayer = willy_prediction_block(
            inFeatures=100, outFeatures=18)

    def forward(self, x):
        # Convolution part
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)

        # fully connected part
        x = self.flatten(x)
        x = self.fullyConnectedLayer(x)
        x = self.predictionLayer(x)
        return F.log_softmax(x,dim=1)
        


# myNet = WillyNet()
# from torchinfo import summary
# summary(myNet)

# Let us compare our model with different pretrained models such as AlexNet and mnasnet0_5

In [ ]:
from torch import nn
from torchvision import models
from torchinfo import summary
import torch
pretrained = models.mnasnet0_5(pretrained=True)

In [1]:
%%writefile WillyMnasNet0_5.py
from torch import nn
from torchvision import models
#from torchinfo import summary
#import torch
#pretrained = models.mnasnet0_5(pretrained=True)

class MyMnasNet0_5(nn.Module):
    def __init__(self, my_pretrained_model=models.mnasnet0_5(pretrained=True)):
        super(MyMnasNet0_5, self).__init__()
        self.pretrained = my_pretrained_model
        self.nr_ftrs = self.pretrained.classifier[1].in_features
        self.pretrained.classifier[1] = nn.Linear(self.nr_ftrs, out_features=18, bias=True)
    def forward(self, x):
        x = self.pretrained(x)
        return nn.functional.log_softmax(x,dim=1)

#willyMnasnet0_5 = MyMnasNet0_5(pretrained)
#summary(willyMnasnet0_5)

Overwriting WillyMnasNet0_5.py


In [ ]:
from WillyMnasNet0_5 import MyMnasNet0_5

willyMnasnet0_5 = MyMnasNet0_5()
for param in willyMnasnet0_5.parameters():
    param.requires_grad = False
#nr_ftr = pretrained.classifier[1].in_features
#pretrained.classifier[1] = nn.Linear(nr_ftr,18)
print(willyMnasnet0_5)
#summary(willyMnasnet0_5)

In [ ]:
from torchinfo import summary
for param in willyMnasnet0_5.pretrained.classifier[-1].parameters():
    param.requires_grad = True
summary(willyMnasnet0_5)

In [2]:
%%writefile WillyAlexNet.py
from torch import nn
from torchvision import models
#from torchinfo import summary
#import torch
#pretrained = models.alexnet(pretrained=True)

class MyAlexNet(nn.Module):
    def __init__(self, my_pretrained_model = models.alexnet(pretrained=True),number_classes=18):
        super(MyAlexNet, self).__init__()
        num_ftrs = my_pretrained_model.classifier[-1].in_features
        my_pretrained_model.classifier[-1] = nn.Linear(num_ftrs, out_features=number_classes, bias=True)
        self.WillyAlexNet = my_pretrained_model

    def forward(self, x):
        x = self.WillyAlexNet(x)
        return nn.functional.log_softmax(x,dim=1)
#willyAlexNet = MyAlexNet()
#print(willyAlexNet)

Overwriting WillyAlexNet.py


In [1]:
from WillyAlexNet import MyAlexNet
from torchinfo import summary

willyAlexNet = MyAlexNet()
print(willyAlexNet)


MyAlexNet(
  (WillyAlexNet): AlexNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
    (classifier): Sequential(
      (0): Dropout(p=0.5, inplace=False)


In [2]:
summary(willyAlexNet, input_size=(5,3,198,198))

Layer (type:depth-idx)                   Output Shape              Param #
MyAlexNet                                --                        --
├─AlexNet: 1-1                           [5, 18]                   --
│    └─Sequential: 2-1                   [5, 256, 5, 5]            --
│    │    └─Conv2d: 3-1                  [5, 64, 48, 48]           23,296
│    │    └─ReLU: 3-2                    [5, 64, 48, 48]           --
│    │    └─MaxPool2d: 3-3               [5, 64, 23, 23]           --
│    │    └─Conv2d: 3-4                  [5, 192, 23, 23]          307,392
│    │    └─ReLU: 3-5                    [5, 192, 23, 23]          --
│    │    └─MaxPool2d: 3-6               [5, 192, 11, 11]          --
│    │    └─Conv2d: 3-7                  [5, 384, 11, 11]          663,936
│    │    └─ReLU: 3-8                    [5, 384, 11, 11]          --
│    │    └─Conv2d: 3-9                  [5, 256, 11, 11]          884,992
│    │    └─ReLU: 3-10                   [5, 256, 11, 11]         

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='1'

from torchvision import models
alexx = models.alexnet(pretrained=True)
summary(alexx,input_size=(5,3,198,198))

NameError: name 'summary' is not defined

In [ ]:
assert torch.cuda.is_available()
device = torch.device('cuda:0')
torch.backends.cudnn.benchmark = True


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='1'

from torchvision import models
from torchinfo import summary
alexx = models.shufflenet_v2_x1_0(pretrained=True)
summary(alexx,input_size=(400,3,198,198))

Layer (type:depth-idx)                   Output Shape              Param #
ShuffleNetV2                             --                        --
├─Sequential: 1-1                        [400, 24, 99, 99]         --
│    └─Conv2d: 2-1                       [400, 24, 99, 99]         648
│    └─BatchNorm2d: 2-2                  [400, 24, 99, 99]         48
│    └─ReLU: 2-3                         [400, 24, 99, 99]         --
├─MaxPool2d: 1-2                         [400, 24, 50, 50]         --
├─Sequential: 1-3                        [400, 116, 25, 25]        --
│    └─InvertedResidual: 2-4             [400, 116, 25, 25]        --
│    │    └─Sequential: 3-1              [400, 58, 25, 25]         1,772
│    │    └─Sequential: 3-2              [400, 58, 25, 25]         5,626
│    └─InvertedResidual: 2-5             [400, 116, 25, 25]        --
│    │    └─Sequential: 3-3              [400, 58, 25, 25]         7,598
│    └─InvertedResidual: 2-6             [400, 116, 25, 25]        --
│    

In [2]:
from torchinfo import summary
from torchvision import models
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

alexx = models.shufflenet_v2_x0_5(pretrained=True)
summary(alexx, input_size=(400, 3, 198, 198))


Layer (type:depth-idx)                   Output Shape              Param #
ShuffleNetV2                             --                        --
├─Sequential: 1-1                        [400, 24, 99, 99]         --
│    └─Conv2d: 2-1                       [400, 24, 99, 99]         648
│    └─BatchNorm2d: 2-2                  [400, 24, 99, 99]         48
│    └─ReLU: 2-3                         [400, 24, 99, 99]         --
├─MaxPool2d: 1-2                         [400, 24, 50, 50]         --
├─Sequential: 1-3                        [400, 48, 25, 25]         --
│    └─InvertedResidual: 2-4             [400, 48, 25, 25]         --
│    │    └─Sequential: 3-1              [400, 24, 25, 25]         888
│    │    └─Sequential: 3-2              [400, 24, 25, 25]         1,512
│    └─InvertedResidual: 2-5             [400, 48, 25, 25]         --
│    │    └─Sequential: 3-3              [400, 24, 25, 25]         1,512
│    └─InvertedResidual: 2-6             [400, 48, 25, 25]         --
│    │ 

In [1]:
import os
from torchvision import models
from torchinfo import summary

os.environ["CUDA_VISIBLE_DEVICES"] = '1'

alexx = models.shufflenet_v2_x1_5(pretrained=False)
summary(alexx, input_size=(400, 3, 198, 198))


Layer (type:depth-idx)                   Output Shape              Param #
ShuffleNetV2                             --                        --
├─Sequential: 1-1                        [400, 24, 99, 99]         --
│    └─Conv2d: 2-1                       [400, 24, 99, 99]         648
│    └─BatchNorm2d: 2-2                  [400, 24, 99, 99]         48
│    └─ReLU: 2-3                         [400, 24, 99, 99]         --
├─MaxPool2d: 1-2                         [400, 24, 50, 50]         --
├─Sequential: 1-3                        [400, 176, 25, 25]        --
│    └─InvertedResidual: 2-4             [400, 176, 25, 25]        --
│    │    └─Sequential: 3-1              [400, 88, 25, 25]         2,552
│    │    └─Sequential: 3-2              [400, 88, 25, 25]         11,176
│    └─InvertedResidual: 2-5             [400, 176, 25, 25]        --
│    │    └─Sequential: 3-3              [400, 88, 25, 25]         16,808
│    └─InvertedResidual: 2-6             [400, 176, 25, 25]        --
│  

In [1]:
import os
from torchvision import models
from torchinfo import summary

os.environ["CUDA_VISIBLE_DEVICES"] = '1'

alexx = models.shufflenet_v2_x2_0(pretrained=False)
summary(alexx, input_size=(400, 3, 198, 198))


Layer (type:depth-idx)                   Output Shape              Param #
ShuffleNetV2                             --                        --
├─Sequential: 1-1                        [400, 24, 99, 99]         --
│    └─Conv2d: 2-1                       [400, 24, 99, 99]         648
│    └─BatchNorm2d: 2-2                  [400, 24, 99, 99]         48
│    └─ReLU: 2-3                         [400, 24, 99, 99]         --
├─MaxPool2d: 1-2                         [400, 24, 50, 50]         --
├─Sequential: 1-3                        [400, 244, 25, 25]        --
│    └─InvertedResidual: 2-4             [400, 244, 25, 25]        --
│    │    └─Sequential: 3-1              [400, 122, 25, 25]        3,436
│    │    └─Sequential: 3-2              [400, 122, 25, 25]        19,642
│    └─InvertedResidual: 2-5             [400, 244, 25, 25]        --
│    │    └─Sequential: 3-3              [400, 122, 25, 25]        31,598
│    └─InvertedResidual: 2-6             [400, 244, 25, 25]        --
│  

In [9]:
from torchinfo import summary
from torchvision import models
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

alexx = models.shufflenet_v2_x0_5(pretrained=True)
summary(alexx, input_size=(400, 3, 198, 198))


Layer (type:depth-idx)                   Output Shape              Param #
ShuffleNetV2                             --                        --
├─Sequential: 1-1                        [400, 24, 99, 99]         --
│    └─Conv2d: 2-1                       [400, 24, 99, 99]         648
│    └─BatchNorm2d: 2-2                  [400, 24, 99, 99]         48
│    └─ReLU: 2-3                         [400, 24, 99, 99]         --
├─MaxPool2d: 1-2                         [400, 24, 50, 50]         --
├─Sequential: 1-3                        [400, 48, 25, 25]         --
│    └─InvertedResidual: 2-4             [400, 48, 25, 25]         --
│    │    └─Sequential: 3-1              [400, 24, 25, 25]         888
│    │    └─Sequential: 3-2              [400, 24, 25, 25]         1,512
│    └─InvertedResidual: 2-5             [400, 48, 25, 25]         --
│    │    └─Sequential: 3-3              [400, 24, 25, 25]         1,512
│    └─InvertedResidual: 2-6             [400, 48, 25, 25]         --
│    │ 

In [12]:
%%writefile WillysShuffleNet_v2_x1_0.py
from torch import nn
from torchvision import models
#from torchinfo import summary
#import torch

class myShuffleNet_v2_x1_0(nn.Module):
    def __init__(self, my_pretrained_model = models.shufflenet_v2_x0_5(pretrained=True),number_classes=18):
        super(myShuffleNet_v2_x1_0, self).__init__()
        num_ftrs = my_pretrained_model.fc.in_features
        my_pretrained_model.fc = nn.Linear(num_ftrs, out_features=number_classes, bias=True)
        self.myShuffleNet_v2_x1_0 = my_pretrained_model

    def forward(self, x):
        x = self.myShuffleNet_v2_x1_0(x)
        return nn.functional.log_softmax(x,dim=1)

#willyAlexNet = myShuffleNet_v2_x1_0()
#print(willyAlexNet)

Writing WillysShuffleNet_v2_x1_0.py


In [11]:
summary(willyAlexNet, input_size=(400, 3, 198, 198))


Layer (type:depth-idx)                        Output Shape              Param #
├─ShuffleNetV2: 1-1                           [400, 18]                 --
│    └─Sequential: 2-1                        [400, 24, 99, 99]         --
│    │    └─Conv2d: 3-1                       [400, 24, 99, 99]         648
│    │    └─BatchNorm2d: 3-2                  [400, 24, 99, 99]         48
│    │    └─ReLU: 3-3                         [400, 24, 99, 99]         --
│    └─MaxPool2d: 2-2                         [400, 24, 50, 50]         --
│    └─Sequential: 2-3                        [400, 48, 25, 25]         --
│    │    └─InvertedResidual: 3-4             [400, 48, 25, 25]         2,400
│    │    └─InvertedResidual: 3-5             [400, 48, 25, 25]         1,512
│    │    └─InvertedResidual: 3-6             [400, 48, 25, 25]         1,512
│    │    └─InvertedResidual: 3-7             [400, 48, 25, 25]         1,512
│    └─Sequential: 2-4                        [400, 96, 13, 13]         --
│    │ 